# Day 13
https://adventofcode.com/2018/day/13

In [1]:
import aocd
data = aocd.get_data(year=2018, day=13)

In [2]:
from dataclasses import dataclass

In [3]:
@dataclass(frozen=True, order=True)
class Point():
    y: int
    x: int
    
    def __add__(self, other):
        return Point(self.y+other.y, self.x+other.x)

In [4]:
def read_tracks(text):
    tracks = {}
    lines = text.translate(text.maketrans('^v<>', '||--')).split('\n')
    for y, line in enumerate(lines):
        for x, char in enumerate(line):
            if char in ('|', '-', '\\', '/', '+'):
                tracks[Point(y, x)] = char
    return tracks

In [5]:
UP = Point(-1, 0)
DOWN = Point(1, 0)
LEFT = Point(0, -1)
RIGHT = Point(0, 1)
COMPASS = (UP, RIGHT, DOWN, LEFT)
TURNS = ('left', 'ahead', 'right')

In [6]:
def turned_direction(direction, change):
    if change == 'ahead':
        return direction
    delta = 1 if change == 'right' else -1
    return COMPASS[(COMPASS.index(direction)+delta) % len(COMPASS)]

In [7]:
@dataclass(frozen=True, order=True)
class Minecart:
    position: Point
    direction: Point
    turns: int
    
    @classmethod
    def create_minecart(cls, y, x, char):
        direction = {
            '^': UP,
            'v': DOWN,
            '<': LEFT,
            '>': RIGHT,
        }.get(char)
        return cls(Point(y, x), direction, 0)
    
    @property
    def next_turn(self):
        return turned_direction(self.direction, TURNS[self.turns % len(TURNS)])
    
    def next_step(self, track_char):
        new_dir = self.direction
        turns = self.turns

        if track_char == '\\':
            if self.direction == UP:
                new_dir = LEFT
            elif self.direction == RIGHT:
                new_dir = DOWN
            elif self.direction == DOWN:
                new_dir = RIGHT
            elif self.direction == LEFT:
                new_dir = UP
        
        elif track_char == '/':
            if self.direction == UP:
                new_dir = RIGHT
            elif self.direction == RIGHT:
                new_dir = UP
            elif self.direction == DOWN:
                new_dir = LEFT
            elif self.direction == LEFT:
                new_dir = DOWN
        
        elif track_char == '+':
            new_dir = self.next_turn
            turns += 1
        
        return Minecart(self.position+new_dir, new_dir, turns)

In [8]:
def read_minecarts(text):
    return [Minecart.create_minecart(y, x, char) for y, line in enumerate(text.split('\n'))
            for x, char in enumerate(line) if char in '^v<>']

In [9]:
def crashes_and_remaining_cart(tracks, minecarts):
    crashes = []
    while len(minecarts) > 1:
        minecarts = sorted(minecarts)
        crashed_this_turn = set()
        for ix in range(len(minecarts)):
            minecarts[ix] = minecarts[ix].next_step(tracks[minecarts[ix].position])
            for crash_position in [minecart.position for minecart in minecarts]:
                victims = [victimid for victimid, minecart in enumerate(minecarts)
                           if minecart.position == crash_position]
                if len(victims) > 1:
                    crashed_this_turn.update(victims)
                    crashes.append(minecarts[victims[0]].position)
        minecarts = [minecart for ix, minecart in enumerate(minecarts) if ix not in crashed_this_turn]
    
    return crashes, minecarts[0]

In [10]:
tracks = read_tracks(data)
minecarts = read_minecarts(data)
crashes, p2 = crashes_and_remaining_cart(tracks, minecarts)
p1 = crashes[0]
print('Part 1: {},{}'.format(p1.x, p1.y))
print('Part 2: {},{}'.format(p2.position.x, p2.position.y))

Part 1: 71,121
Part 2: 71,76
